In [1]:
import random as rn
import pickle
from datetime import datetime
import pandas as pd
from hs_gimme.db_facade.db_facade_factory import get_mongo_client_db
from hs_tree_blenders_creation.batch_runners.data_sets import get_account_test_set, get_account_train_set
from hs_gimme.application_status_history_classifier.machine_learning_status_classifier import \
    get_machine_learning_status_classifier
from tqdm import tqdm

In [2]:
ACCOUNTS = ['utah', 'jamaica', 'dallas', 'indiana']

In [3]:
ENV = 'production'
TEST_SIZE_PER_ACCOUNT = 1000
NOW = datetime.now()

In [4]:
def get_data_for_req(req, mongo, status_cls):
    
    result = []
    
    apps = list(mongo.application.find({'req_id': req['_id']}))
    
    req_data = []
    for app in apps:
        max_status = status_cls.get_max_status(app)
        days = (app['date_applied'] - req['job_create_date']).days

        req_data.append({'days': days, 'max_status': max_status})

    df = pd.DataFrame(req_data)
    
    if len(df) > 0:
        result = []
        for days in range(df['days'].max()):
            result.append({
                'req_id': req['_id'],
                'days': days,
                'req': req,
                'num_of_apps': len(df[df['days'] <= days]),
                'num_of_interview': len(df[(df['days'] <= days) & (df['max_status'] >= 2)]),
                'num_of_offer': len(df[(df['days'] <= days) & (df['max_status'] >= 3)]),
            })
    return result

In [5]:
data = []

for account in tqdm(ACCOUNTS):
    req_ids = list(get_account_train_set(account))
    
    req_ids = rn.sample(req_ids, min(len(req_ids), TEST_SIZE_PER_ACCOUNT))
    
    mongo = get_mongo_client_db(ENV, account)
    status_cls = get_machine_learning_status_classifier(ENV, account)
    
    for req in mongo.req.find({'_id': {'$in': req_ids}}):
        data += get_data_for_req(req, mongo, status_cls)

pickle.dump(data, open('job_intel_hack_data_train.pkl', 'wb'))

  0%|                                                                                                                                                                                     | 0/4 [00:00<?, ?it/s]

2023-06-27T13:09:19.732130Z [info     ] Getting model reader           environment=production file_path=/Users/dima/hiredscore/brain/brainme/hs_brainme/models/model_persistence/models_persistency.py function_name=get_model_reader hostname=Dima-Shulga-MacBook-Pro line_number=82 model_key=utah model_manager=hs_grading model_name=data_sets model_type=pickle models_reader=LocalPickleModelsReader module=hs_brainme.models.model_persistence.models_persistency pid=24333
2023-06-27T13:09:19.733508Z [info     ] Loading model file             compressed__boolean=True dir=/Users/dima/hiredscore_model_manager/hs_grading/data_sets/utah_20220915130441 environment=production file_name=/Users/dima/hiredscore_model_manager/hs_grading/data_sets/utah_20220915130441/key.pickle file_path=/Users/dima/hiredscore/brain/brainme/hs_brainme/models/model_persistence/models_persistency.py function_name=load hostname=Dima-Shulga-MacBook-Pro key=key line_number=167 model_type=pickle module=hs_brainme.models.model_per

  0%|                                                                                                                                                                                     | 0/4 [18:54<?, ?it/s]


CursorNotFound: cursor id 42248938080052 not found, full error: {'operationTime': Timestamp(1687872494, 24), 'ok': 0.0, 'errmsg': 'cursor id 42248938080052 not found', 'code': 43, 'codeName': 'CursorNotFound', '$clusterTime': {'clusterTime': Timestamp(1687872494, 24), 'signature': {'hash': b'-\xed\xe8\x93+\xa7G\xd6^k\xaa\x97\xb3<\xc2\xfe*t\xd6]', 'keyId': 7201094772956594370}}}